# EEG Biometrics


In [33]:
# Run this cell to load required libraries and mount your Drive folder

import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive
import os


drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
# Run this cell to navigate through the Drive filesystem and retrieve the dataset

dataset_path = '/content/drive/MyDrive/ml2-eeg-biometrics/raw-data/'
example_file_name = '/content/drive/MyDrive/ml2-eeg-biometrics/raw-data/A1/A1_20190613/NSsignal_2019_06_13_19_16_33.npz'

sample_rate = 500 #How many samples per second
task_duration = 9 # Task average duration is 9 seconds

dataset = {}

folder_list = os.listdir(dataset_path)

# Navigate directories and files in the dataset
for participant_folder in folder_list:
  #print("Participant " + participant_folder)
  dataset[participant_folder] = []
  for folder in os.listdir(dataset_path + participant_folder):
    #print("Folder: " + folder)
    date = folder.split("_")[1]
    dataset[participant_folder].append(date)
    for file in os.listdir(dataset_path + participant_folder + "/" + folder):
      #print(file)
      foo = 0

print(dataset)

npz_data = np.load(example_file_name)

npz_data_dict = dict(npz_data)
fs = npz_data_dict['SampleRate'][0]
index = npz_data_dict['MarkOnSignal'] 
data = npz_data_dict['signal']

print(data)


{'S1': ['20200718', '20200719', '20200720', '20200721', '20200722', '20200723', '20200724'], 'S2': ['20200718', '20200719', '20200720', '20200721', '20200722', '20200723', '20200724'], 'S3': ['20200718', '20200721', '20200719', '20200720', '20200722', '20200723', '20200724'], 'S4': ['20200718', '20200719', '20200720', '20200721', '20200722', '20200723', '20200724'], 'S5': ['20200724', '20200720', '20200722', '20200719', '20200718', '20200723', '20200721'], 'S6': ['20200720', '20200718', '20200719', '20200724', '20200722', '20200723', '20200721'], 'S12': ['20200728', '20200727', '20200730', '20200801', '20200731', '20200726', '20200729'], 'S11': ['20200801', '20200731', '20200726', '20200729', '20200730', '20200727', '20200728'], 'S7': ['20200727', '20200730', '20200731', '20200728', '20200726', '20200729', '20200801'], 'S8': ['20200726', '20200727', '20200728', '20200729', '20200730', '20200731', '20200801'], 'S9': ['20200726', '20200727', '20200728', '20200801', '20200729', '20200730'

In [ ]:
# Run this test to load a dataset with Numpy

with open(example_file_name, 'w') as dataset_file:
  print(dataset_file)
  data=np.load(dataset_file) 
  print(data)

In [ ]:
# Run this cell to save the changes

drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')